# Project 2

    In order to better understand how our organization can finance the investments in the Video game industry we previously analyzed, we must first understand the state of our organization.

 

    You have been charged with analyzing the current state of our organization to determine what level of investment we can make in the game industry. To accomplish this you must do the following:

 

    1) How have video game sales performed in our organization as a whole?
        Show the profit for media and gaming for our entire company by month.
    2) What do you feel is the direct cause of the performance of media and gaming?
    3) After evaluating the media and gaming department in our company, what is your recommendation for investment in the video game industry?

In [2]:
from dbConfig import create_db_engine, execute_query
import pandas as pd

# Test connection and get basic info about sales table
engine = create_db_engine()

# Method 1: Using the execute_query helper function
df = execute_query("SELECT COUNT(*) as total_sales FROM sales")
print("Total sales records:", df.iloc[0]['total_sales'])

# Method 2: Direct pandas query (alternative approach)
# df = pd.read_sql("SELECT COUNT(*) as total_sales FROM sales", engine)
# print(df)

ModuleNotFoundError: No module named 'mysql'